In [1]:
from Sindonews import Sindonews #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [5]:
obj = Sindonews()
end_date = date(2018, 7, 24)
start_date = date(2018, 5, 7)
init_page = 14
init_offset = 130

In [ ]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, 0, init_offset, single_date.strftime("%Y-%m-%d"))
    init_page = 1
    init_offset = 0

2018-05-09
page  1
https://international.sindonews.com/read/1304449/40/serangan-terkoordinasi-hantam-ibu-kota-afghanistan-1525884947
Insert berita  Serangan Terkoordinasi Hantam Ibu Kota Afghanistan
masuk
https://daerah.sindonews.com/read/1304448/21/pasangan-rindu-dukung-pemekaran-subang-utara-1525883922
Insert berita  Pasangan Rindu Dukung Pemekaran Subang Utara
masuk
https://ekbis.sindonews.com/read/1304445/178/ciputra-life-luncurkan-jalur-distribusi-digital-1525882750
Insert berita  Ciputra Life Luncurkan Jalur Distribusi Digital
masuk
https://autotekno.sindonews.com/read/1304447/123/hello-yo-ajak-fans-mobile-legends-main-game-sambil-beramal-1525883411
Insert berita  Hello Yo Ajak Fans Mobile Legends Main Game Sambil Beramal
masuk
https://lifestyle.sindonews.com/read/1304398/155/haid-tak-teratur-bisa-menjadi-risiko-terkena-diabetes-tipe-2-1525871974
Insert berita  Haid Tak Teratur Bisa Menjadi Risiko Terkena Diabetes Tipe 2
masuk
https://international.sindonews.com/read/1304446/40/m